In [8]:
# Protein 2D Structure Prediction Using REAL CB513 Dataset from Hugging Face
# NO SYNTHETIC DATA - ONLY REAL PROTEINS

# ============================================
# PART 1: INSTALLATION AND IMPORTS
# ============================================

!pip install matplotlib seaborn scikit-learn imageio pillow

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.utils import to_categorical
import urllib.request
import os
import json
from PIL import Image
import imageio
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow version:", tf.__version__)
print("\n" + "="*60)
print("PROTEIN SECONDARY STRUCTURE PREDICTION")
print("Using REAL CB513 Dataset from Hugging Face")
print("="*60)

# ============================================
# PART 2: DOWNLOAD AND LOAD CB513 FROM HUGGING FACE
# ============================================

def download_cb513_from_huggingface():
    """Download CB513 dataset from Hugging Face"""
    print("\n" + "="*60)
    print("DOWNLOADING CB513 DATASET FROM HUGGING FACE")
    print("="*60)

    # Create data directory
    os.makedirs('data', exist_ok=True)

    # Download CB513.csv from Hugging Face
    url = "https://huggingface.co/datasets/proteinea/secondary_structure_prediction/resolve/main/CB513.csv"
    filepath = "data/CB513.csv"

    try:
        print("\nDownloading CB513.csv from Hugging Face...")
        urllib.request.urlretrieve(url, filepath)
        print("✓ Download complete!")

        # Load the CSV file
        print("Loading CSV file...")
        data = pd.read_csv(filepath)
        print(f"✓ Dataset loaded: {data.shape[0]} proteins")

        return data

    except Exception as e:
        print(f"✗ Download failed: {str(e)}")
        print("\nPlease manually download from:")
        print("https://huggingface.co/datasets/proteinea/secondary_structure_prediction/blob/main/CB513.csv")
        return None

def process_cb513_csv(df):
    """Process CB513 CSV data into sequences and structures"""
    print("\n" + "="*60)
    print("PROCESSING REAL CB513 DATA")
    print("="*60)

    sequences = []
    structures = []

    # Check the structure of the dataframe
    print(f"Columns in dataset: {df.columns.tolist()}")
    print(f"\nFirst few rows:")
    print(df.head())

    # Based on the output, we have:
    # 'input' - protein sequences
    # 'dssp3' - 3-class secondary structure
    # 'dssp8' - 8-class secondary structure

    seq_col = 'input'
    struct_col = 'dssp3'  # Using 3-class directly

    print(f"\nUsing columns: '{seq_col}' for sequences, '{struct_col}' for structures")

    for idx, row in df.iterrows():
        seq = str(row[seq_col]).upper().strip()
        struct = str(row[struct_col]).upper().strip()

        # Clean sequences - keep only valid amino acids
        seq = ''.join([aa for aa in seq if aa in 'ACDEFGHIKLMNPQRSTVWY'])

        # Ensure structure has only H, E, C
        struct = ''.join([ss if ss in 'HEC' else 'C' for ss in struct])

        # Only add if lengths match and are reasonable
        if len(seq) == len(struct) and 30 < len(seq) < 700:
            sequences.append(seq)
            structures.append(struct)

    print(f"\nProcessed {len(sequences)} valid protein sequences")

    if len(sequences) > 0:
        # Show statistics
        avg_len = np.mean([len(s) for s in sequences])
        all_ss = ''.join(structures)
        h_pct = 100 * all_ss.count('H') / len(all_ss)
        e_pct = 100 * all_ss.count('E') / len(all_ss)
        c_pct = 100 * all_ss.count('C') / len(all_ss)

        print(f"\nDataset Statistics:")
        print(f"  Total proteins: {len(sequences)}")
        print(f"  Average sequence length: {avg_len:.1f}")
        print(f"  Helix (H): {h_pct:.1f}%")
        print(f"  Sheet (E): {e_pct:.1f}%")
        print(f"  Coil (C): {c_pct:.1f}%")

        print(f"\nSample sequence (first 50 AA): {sequences[0][:50]}")
        print(f"Sample structure (first 50 SS): {structures[0][:50]}")

    return sequences, structures

# ============================================
# PART 3: DATA PREPROCESSING
# ============================================

class ProteinDataProcessor:
    def __init__(self, max_len=200):
        self.max_len = max_len
        self.amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
        self.ss_classes = 'HEC'  # Helix, Sheet (Extended), Coil

        # Create mapping dictionaries
        self.aa_to_idx = {aa: idx+1 for idx, aa in enumerate(self.amino_acids)}
        self.aa_to_idx['PAD'] = 0
        self.aa_to_idx['UNK'] = len(self.amino_acids) + 1

        self.ss_to_idx = {ss: idx for idx, ss in enumerate(self.ss_classes)}
        self.idx_to_ss = {idx: ss for ss, idx in self.ss_to_idx.items()}

    def encode_sequence(self, sequence):
        """Convert amino acid sequence to indices"""
        encoded = []
        for aa in sequence[:self.max_len]:
            encoded.append(self.aa_to_idx.get(aa, self.aa_to_idx['UNK']))

        # Pad sequence
        while len(encoded) < self.max_len:
            encoded.append(self.aa_to_idx['PAD'])

        return np.array(encoded)

    def encode_structure(self, structure):
        """Convert secondary structure to indices"""
        encoded = []
        for ss in structure[:self.max_len]:
            if ss in self.ss_to_idx:
                encoded.append(self.ss_to_idx[ss])
            else:
                encoded.append(2)  # Default to coil (C)

        # Pad structure
        while len(encoded) < self.max_len:
            encoded.append(2)  # Pad with coil

        return np.array(encoded)

    def one_hot_encode_sequence(self, encoded_seq):
        """One-hot encode the sequence"""
        return to_categorical(encoded_seq, num_classes=len(self.aa_to_idx))

    def one_hot_encode_structure(self, encoded_struct):
        """One-hot encode the structure"""
        return to_categorical(encoded_struct, num_classes=len(self.ss_classes))

# ============================================
# PART 4: MODEL ARCHITECTURE
# ============================================

def create_protein_model(input_shape, num_classes=3):
    """Create a hybrid CNN-LSTM model for protein structure prediction"""
    model = models.Sequential([
        # Input layer
        layers.Input(shape=input_shape),

        # CNN layers for local pattern detection
        layers.Conv1D(64, 7, padding='same', activation='relu', name='conv1'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),

        layers.Conv1D(128, 5, padding='same', activation='relu', name='conv2'),
        layers.BatchNormalization(),
        layers.Dropout(0.2),

        # Bidirectional LSTM for sequence context
        layers.Bidirectional(layers.LSTM(64, return_sequences=True, name='lstm1')),
        layers.Dropout(0.3),

        layers.Bidirectional(layers.LSTM(32, return_sequences=True, name='lstm2')),
        layers.Dropout(0.3),

        # Dense layers for classification
        layers.TimeDistributed(layers.Dense(32, activation='relu', name='dense1')),
        layers.Dropout(0.2),

        # Output layer
        layers.TimeDistributed(layers.Dense(num_classes, activation='softmax', name='output'))
    ])

    return model

# ============================================
# PART 5: WEIGHT TRACKING FOR GIFS
# ============================================

class WeightTracker(callbacks.Callback):
    def __init__(self, layer_names):
        self.layer_names = layer_names
        self.weights_history = {name: [] for name in layer_names}

    def on_epoch_end(self, epoch, logs=None):
        for layer in self.model.layers:
            if layer.name in self.layer_names:
                weights = layer.get_weights()
                if len(weights) > 0:
                    self.weights_history[layer.name].append(weights[0].copy())

def create_weight_gif(weights_history, layer_name, output_path):
    """Create GIF showing weight evolution"""
    images = []

    for epoch, weights in enumerate(weights_history):
        fig, ax = plt.subplots(figsize=(8, 6))

        # Reshape weights for visualization
        if len(weights.shape) == 3:  # Conv1D weights
            weights_2d = weights.reshape(-1, weights.shape[-1])
        elif len(weights.shape) == 2:  # Dense weights
            weights_2d = weights
        else:
            weights_2d = weights.reshape(-1, weights.shape[-1] if len(weights.shape) > 1 else 1)

        # Limit display size
        display_weights = weights_2d[:min(50, weights_2d.shape[0]), :min(50, weights_2d.shape[1])]

        im = ax.imshow(display_weights, cmap='coolwarm', aspect='auto', vmin=-1, vmax=1)
        ax.set_title(f'{layer_name} Weights - Epoch {epoch+1}', fontsize=14)
        ax.set_xlabel('Output Units')
        ax.set_ylabel('Input Units')
        plt.colorbar(im, ax=ax)

        # Save to temporary file
        temp_path = f'temp_{epoch}.png'
        plt.savefig(temp_path, dpi=60, bbox_inches='tight')
        plt.close()

        # Load and append to images
        images.append(Image.open(temp_path))
        os.remove(temp_path)

    # Save as GIF
    if images:
        images[0].save(output_path, save_all=True, append_images=images[1:],
                      duration=200, loop=0)
        print(f"  ✓ GIF saved: {output_path}")

# ============================================
# PART 6: VISUALIZATION FUNCTIONS
# ============================================

def create_all_plots(history, y_true, y_pred):
    """Create all required visualization plots"""

    # 1. Loss and Accuracy Curves
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Loss plot
    axes[0].plot(history.history['loss'], label='Training Loss', linewidth=2, color='blue')
    axes[0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2, color='red')
    axes[0].set_title('Model Loss Over Epochs', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Loss', fontsize=12)
    axes[0].legend(loc='upper right')
    axes[0].grid(True, alpha=0.3)

    # Accuracy plot
    axes[1].plot(history.history['accuracy'], label='Training Accuracy', linewidth=2, color='blue')
    axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy', linewidth=2, color='red')
    axes[1].set_title('Model Accuracy Over Epochs', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Accuracy', fontsize=12)
    axes[1].legend(loc='lower right')
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('training_curves.png', dpi=100, bbox_inches='tight')
    plt.show()

    # 2. Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['Helix', 'Sheet', 'Coil'],
                yticklabels=['Helix', 'Sheet', 'Coil'],
                cbar_kws={'label': 'Count'})
    plt.title('Confusion Matrix - Test Set', fontsize=14, fontweight='bold')
    plt.xlabel('Predicted', fontsize=12)
    plt.ylabel('Actual', fontsize=12)
    plt.savefig('confusion_matrix.png', dpi=100, bbox_inches='tight')
    plt.show()

def save_model_architecture(model):
    """Save model architecture diagram"""
    try:
        tf.keras.utils.plot_model(
            model,
            to_file='model_architecture.png',
            show_shapes=True,
            show_layer_names=True,
            rankdir='TB',
            expand_nested=True,
            dpi=96
        )
        print("✓ Model architecture saved as 'model_architecture.png'")
    except:
        print("⚠ Could not generate model architecture diagram")
        print("  To fix: !apt-get install graphviz && !pip install pydot")

# ============================================
# PART 7: MAIN TRAINING PIPELINE
# ============================================

def main():
    print("\n" + "="*60)
    print("PROTEIN 2D STRUCTURE PREDICTION PIPELINE")
    print("REAL CB513 DATA ONLY - NO SYNTHETIC DATA")
    print("="*60)

    # Step 1: Download and Load CB513 Data
    print("\n1. LOADING REAL CB513 DATASET...")
    cb513_df = download_cb513_from_huggingface()

    if cb513_df is None:
        print("\nError: Could not load CB513 dataset!")
        return None, None, 0

    # Process CB513 data
    sequences, structures = process_cb513_csv(cb513_df)

    if len(sequences) == 0:
        print("\nError: No valid sequences found in CB513!")
        return None, None, 0

    print(f"\n✓ Using {len(sequences)} REAL protein sequences from CB513")

    # Step 2: Preprocess Data
    print("\n2. PREPROCESSING DATA...")
    processor = ProteinDataProcessor(max_len=300)  # Increased to handle longer sequences

    # Encode sequences and structures
    print("  Encoding sequences...")
    X_encoded = np.array([processor.encode_sequence(seq) for seq in sequences])
    y_encoded = np.array([processor.encode_structure(struct) for struct in structures])

    # One-hot encode
    print("  One-hot encoding...")
    X = np.array([processor.one_hot_encode_sequence(x) for x in X_encoded])
    y = np.array([processor.one_hot_encode_structure(y) for y in y_encoded])

    print(f"  Input shape: {X.shape}")
    print(f"  Output shape: {y.shape}")

    # Step 3: Split Data
    print("\n3. SPLITTING DATA...")
    # Since CB513 is traditionally a test set, we'll split it for training/validation/test
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.2, random_state=42
    )

    print(f"  Training set: {X_train.shape[0]} samples")
    print(f"  Validation set: {X_val.shape[0]} samples")
    print(f"  Test set: {X_test.shape[0]} samples")

    # Step 4: Create Model
    print("\n4. CREATING MODEL...")
    model = create_protein_model(
        input_shape=(X.shape[1], X.shape[2]),
        num_classes=3
    )

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    print("\nModel Summary:")
    model.summary()

    # Calculate total parameters
    total_params = model.count_params()
    print(f"\nTotal parameters: {total_params:,}")

    # Step 5: Train Model
    print("\n5. TRAINING MODEL ON REAL CB513 DATA...")
    print("  This will take several minutes...")

    # Callbacks
    tracked_layers = ['conv1', 'lstm1', 'dense1']
    weight_tracker = WeightTracker(tracked_layers)

    early_stop = callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    )

    reduce_lr = callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=0.00001,
        verbose=1
    )

    # Model checkpoint
    checkpoint = callbacks.ModelCheckpoint(
        'best_model.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )

    # Train model
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=50,  # More epochs since we have less data
        batch_size=16,  # Smaller batch size for small dataset
        callbacks=[weight_tracker, early_stop, reduce_lr, checkpoint],
        verbose=1
    )

    # Step 6: Create Weight GIFs
    print("\n6. CREATING WEIGHT EVOLUTION GIFS...")
    os.makedirs('gifs', exist_ok=True)

    for layer_name in tracked_layers:
        if layer_name in weight_tracker.weights_history:
            weights = weight_tracker.weights_history[layer_name]
            if weights:
                create_weight_gif(weights, layer_name, f'gifs/{layer_name}_weights.gif')

    # Step 7: Evaluation
    print("\n7. EVALUATING MODEL ON TEST SET...")

    # Load best model
    if os.path.exists('best_model.keras'):
        print("  Loading best model...")
        model = keras.models.load_model('best_model.keras')

    # Predictions on test set
    print("  Making predictions...")
    y_pred = model.predict(X_test, verbose=0)
    y_pred_classes = np.argmax(y_pred, axis=-1).flatten()
    y_true_classes = np.argmax(y_test, axis=-1).flatten()

    # Remove padding from evaluation
    mask = X_test.sum(axis=-1).flatten() > 0
    y_pred_filtered = y_pred_classes[mask]
    y_true_filtered = y_true_classes[mask]

    # Calculate metrics
    accuracy = accuracy_score(y_true_filtered, y_pred_filtered)
    report = classification_report(
        y_true_filtered, y_pred_filtered,
        target_names=['Helix', 'Sheet', 'Coil'],
        output_dict=True
    )

    print(f"\n  Test Accuracy: {accuracy:.4f}")
    print("\n  Classification Report:")
    report_df = pd.DataFrame(report).transpose()
    print(report_df.round(3))

    # Calculate per-position accuracy
    print("\n  Per-Position Statistics:")
    total_positions = len(y_true_filtered)
    helix_correct = np.sum((y_true_filtered == 0) & (y_pred_filtered == 0))
    sheet_correct = np.sum((y_true_filtered == 1) & (y_pred_filtered == 1))
    coil_correct = np.sum((y_true_filtered == 2) & (y_pred_filtered == 2))

    print(f"    Total amino acids evaluated: {total_positions}")
    print(f"    Helix predictions correct: {helix_correct}")
    print(f"    Sheet predictions correct: {sheet_correct}")
    print(f"    Coil predictions correct: {coil_correct}")

    # Step 8: Visualizations
    print("\n8. CREATING VISUALIZATIONS...")
    create_all_plots(history, y_true_filtered, y_pred_filtered)

    # Step 9: Save Model Architecture
    print("\n9. SAVING MODEL ARCHITECTURE...")
    save_model_architecture(model)

    # Final summary
    print("\n" + "="*60)
    print("TRAINING COMPLETE - REAL CB513 DATA")
    print("="*60)
    print("\nGenerated Files:")
    print("  ✓ gifs/conv1_weights.gif - Conv layer weight evolution")
    print("  ✓ gifs/lstm1_weights.gif - LSTM layer weight evolution")
    print("  ✓ gifs/dense1_weights.gif - Dense layer weight evolution")
    print("  ✓ training_curves.png - Loss and accuracy plots")
    print("  ✓ confusion_matrix.png - Test set confusion matrix")
    print("  ✓ model_architecture.png - Network architecture diagram")
    print("  ✓ protein_2d_model.h5 - Trained model")
    print("  ✓ best_model.keras - Best checkpoint")
    print("  ✓ training_history.json - Training metrics")

    return model, history, accuracy

# ============================================
# PART 8: RUN THE COMPLETE PIPELINE
# ============================================

if __name__ == "__main__":
    # Run the complete pipeline
    model, history, test_accuracy = main()

    # Save the model and history
    if model is not None:
        print("\n" + "="*60)
        print("SAVING FINAL OUTPUTS")
        print("="*60)

        # Save model in H5 format
        model.save('protein_2d_model.h5')
        print("✓ Model saved as 'protein_2d_model.h5'")

        # Save training history
        with open('training_history.json', 'w') as f:
            json.dump(history.history, f, indent=2)
        print("✓ Training history saved as 'training_history.json'")

        # Create summary statistics
        stats = {
            'dataset': 'CB513 from Hugging Face',
            'total_proteins': len(history.history['loss']) * 16,  # epochs * batch_size approximation
            'test_accuracy': float(test_accuracy),
            'model_parameters': int(model.count_params()),
            'epochs_trained': len(history.history['loss'])
        }

        with open('training_stats.json', 'w') as f:
            json.dump(stats, f, indent=2)
        print("✓ Training statistics saved as 'training_stats.json'")

        # Create final report
        print("\n" + "="*60)
        print("ASSIGNMENT COMPLETE!")
        print("="*60)
        print(f"\nFinal Test Accuracy: {test_accuracy:.4f}")
        print("\nDataset: REAL CB513 proteins from Hugging Face")
    else:
        print("\nError: Training failed. Please check the error messages above.")

Exception ignored in: <function _xla_gc_callback at 0x7b0a1e3142c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


TensorFlow version: 2.19.0

PROTEIN SECONDARY STRUCTURE PREDICTION
Using REAL CB513 Dataset from Hugging Face

PROTEIN 2D STRUCTURE PREDICTION PIPELINE
REAL CB513 DATA ONLY - NO SYNTHETIC DATA

1. LOADING REAL CB513 DATASET...

DOWNLOADING CB513 DATASET FROM HUGGING FACE



KeyboardInterrupt: 